# Generate automated session reports for all participants

In [1]:
import pandas as pd
import numpy as np
import mne
import mne_bids
#from matplotlib import pyplot as plt
#import plotly.express as px
#import plotly.graph_objects as go
#import plotly.offline as py
#import plotly.io as pio
#pio.renderers.default = "plotly_mimetype+notebook"
#import q1k_init_tools as qit
import glob
import warnings
#from nbconvert import HTMLExporter
#from nbconvert.preprocessors import ExecutePreprocessor
#from nbformat import read

import re
import subprocess

import os
import papermill as pm
warnings.filterwarnings('ignore')

#from matplotlib import pyplot as plt
#import plotly.express as px
#import plotly.graph_objects as go
#import plotly.offline as py
#import plotly.io as pio
#pio.renderers.default = "plotly_mimetype+notebook"
from nbconvert import HTMLExporter
from nbconvert.preprocessors import ExecutePreprocessor
from nbformat import read


### Select task parameters and file paths

In [2]:
# Select kind of data experimental or control group

dataset_group = "experimental"

if dataset_group == "control":
    # Control group data
    project_path = "/home/james/q1k/pilot/q1k-external-pilot/"
    task_id_in = "ap"
    task_id_in_et = "ap"
    task_id_out = "ap"
    #subject_id = '002'
    session_id = '01'
    run_id = '1'

elif dataset_group == "experimental":
# Experimental group data

    project_path = "/project/def-emayada/q1k/experimental/hsj/"
    pyll_path = "derivatives/pylossless/"
    post_path = "derivatives/postproc/"
    out_seg_path = "epoch_fif_files/"
    out_avg_path = "erp_fif_files/"
    html_reports_path = "session_reports/"
    task_id_in = "VEP"
    task_id_in_et = "VEP"
    task_id_out = "VEP"
    #subject_id = 'Q1K_HSJ_100123_F1'
    run_id = '1'
    session_id = '01'
    #sourcedata_path = "sourcedata/" 
    #html_reports_path = "session_reports/" + task_id_in  + '/'



In [3]:
# Sanity check to see which task you would like to make reports for 
print(task_id_in)

VEP


In [4]:
# Function to extract details from filename
def extract_job_info(filename):
    # Define the pattern to capture the required sections
    pattern = r"sub-(.*?)_ses-(.*?)_task-(.*?)_run-(.*?)_eeg\.edf"
    match = re.match(pattern, filename)

    if match:
        # Extract the groups from the match
        subject_id = match.group(1)
        session_id = match.group(2)
        task_id = match.group(3)
        run_id = match.group(4)
        return subject_id, session_id, task_id, run_id
    else:
        raise ValueError("Filename pattern did not match.")


in_pattern = project_path + pyll_path + '**/eeg/*' + task_id_out + '*.edf'
in_file_paths = glob.glob(in_pattern, recursive=True)
print(f"Input files:")
for file_path in in_file_paths:
    file_name = os.path.basename(file_path)  # Extract the filename from the full path
    print(f"File Path: {file_path}, File Name: {file_name}")

    
# Make sure the output directories exists
if not os.path.exists(project_path + pyll_path + post_path + out_seg_path + task_id_out):
    os.makedirs(project_path + pyll_path + post_path + out_seg_path + task_id_out)
if not os.path.exists(project_path + pyll_path + post_path + out_avg_path + task_id_out):
    os.makedirs(project_path + pyll_path + post_path + out_avg_path + task_id_out)
    
out_seg_pattern = project_path + pyll_path + post_path + out_seg_path + task_id_out + '/*' + task_id_out + '*epo.fif'
out_seg_file_paths = glob.glob(out_seg_pattern, recursive=True)
print(f"existing output epoch files:")
for file_path in out_seg_file_paths:
    file_name = os.path.basename(file_path)  # Extract the filename from the full path
    print(f"File Path: {file_path}, File Name: {file_name}")

out_avg_pattern = project_path + pyll_path + post_path + out_avg_path + task_id_out + '/*' + task_id_out + '*epo.fif'
out_avg_file_paths = glob.glob(out_seg_pattern, recursive=True)
print(f"existing output avg files:")
for file_path in out_avg_file_paths:
    file_name = os.path.basename(file_path)  # Extract the filename from the full path
    print(f"File Path: {file_path}, File Name: {file_name}")

    #submit_slurm_job(project_path, file_name)

Input files:
File Path: /project/def-emayada/q1k/experimental/hsj/derivatives/pylossless/sub-100114M1/ses-01/eeg/sub-100114M1_ses-01_task-VEP_run-1_eeg.edf, File Name: sub-100114M1_ses-01_task-VEP_run-1_eeg.edf
File Path: /project/def-emayada/q1k/experimental/hsj/derivatives/pylossless/sub-10251061M/ses-01/eeg/sub-10251061M_ses-01_task-VEP_run-1_eeg.edf, File Name: sub-10251061M_ses-01_task-VEP_run-1_eeg.edf
File Path: /project/def-emayada/q1k/experimental/hsj/derivatives/pylossless/sub-100100F1/ses-01/eeg/sub-100100F1_ses-01_task-VEP_run-1_eeg.edf, File Name: sub-100100F1_ses-01_task-VEP_run-1_eeg.edf
File Path: /project/def-emayada/q1k/experimental/hsj/derivatives/pylossless/sub-100146P/ses-01/eeg/sub-100146P_ses-01_task-VEP_run-1_eeg.edf, File Name: sub-100146P_ses-01_task-VEP_run-1_eeg.edf
File Path: /project/def-emayada/q1k/experimental/hsj/derivatives/pylossless/sub-15251026P/ses-01/eeg/sub-15251026P_ses-01_task-VEP_run-1_eeg.edf, File Name: sub-15251026P_ses-01_task-VEP_run-1_ee

## Generate html session reports for all participants

In [5]:
# Directory to save HTML reports
#html_reports_dir = './../../session_reports/' + task_id_in + '/'


# Create a list for participants with errors
error_subjects = []

#for file in glob.glob("./../../sourcedata/eeg/Q1K*/Q1K" + "*AEP" + '*.mff'):
#for file in glob.glob(project_path + sourcedata_path + "eeg/Q1K*/Q1K*" + task_id_in + '*.mff'):
for file in in_file_paths:
    
    file_name = os.path.basename(file)
    print('Current data file: ' + file_name)
    ## Select anything after the Q1K and before the AEP
    ##subject_id = file.split('_')[2]
    ##site_part = file.split("HSJ_")
    subject_id = file_name.split("_")[0].split("-")[1]
    print('Participant ID: ' + subject_id)

    #subject_relation = site_part[1].split("_")[1].split("/")[0]
    #print('Participant relation: ' + subject_relation)

    #subject_id_in = subject_number + "_" + subject_relation
    #print('Participant ID input: ' + subject_id_in)
    #subject_id_out = subject_number.replace('_','').replace('-','') + subject_relation
    #print('Participant ID output: ' + subject_id_out)
    
    # Skip sessions that have already been processed
    print(subject_id)
    if subject_id in out_seg_file_paths:
        print(subject_id + ' has already been processed')
        continue    

    # Define paths
    input_notebook = project_path + pyll_path + post_path + 'code/q1k_postproc_erp/' + 'session_seg_' + task_id_in + '.ipynb'

    ##output_notebook = f'./../../session_reports/{task_id_in}/executed_notebooks/{subject_id}_{task_id_in}_executed.ipynb'
    ##output_html = f'./../../session_reports/{task_id_in}/{subject_id}_{session_id}_{task_id_in}.html'
    ##output_html = f'{html_reports_dir}/{subject_id}.html'
    output_notebook = f'{project_path}{pyll_path}{post_path}{html_reports_path}executed_notebooks/{subject_id}_{task_id_in}_executed.ipynb'
    print('Output notebook file: ' + output_notebook)
    output_html = f'{project_path}{pyll_path}{post_path}{html_reports_path}{task_id_out}/{subject_id}_{task_id_in}.html'
    print('Output HTML file: ' + output_html)

    try:
        # Execute the notebook
        pm.execute_notebook(input_notebook, output_notebook, kernel_name = 'q1k_postproc_erp', parameters=dict(subject_id=subject_id, task_id=task_id_in,
        session_id=session_id, run_id=run_id))

        # Convert executed notebook to HTML
        html_exporter = HTMLExporter()
        html_exporter.exclude_input = True

        (body, resources) = html_exporter.from_filename(output_notebook)

        # Save HTML output
        with open(output_html, 'w', encoding='utf-8') as f:
            f.write(body)

        print(f"HTML report saved for {subject_id}.")
    

    except Exception as e:
        # Handle the error 
        error_subjects.append(subject_id)
        print(f"Error while processing {subject_id}: {e}")

        # Print out the list of subjects with errors
print( "These subjects have errors: " + str(error_subjects) + " and need to be reprocessed")

Current data file: sub-100114M1_ses-01_task-VEP_run-1_eeg.edf
Participant ID: 100114M1
100114M1
Output notebook file: /project/def-emayada/q1k/experimental/hsj/derivatives/pylossless/derivatives/postproc/session_reports/executed_notebooks/100114M1_VEP_executed.ipynb
Output HTML file: /project/def-emayada/q1k/experimental/hsj/derivatives/pylossless/derivatives/postproc/session_reports/VEP/100114M1_VEP.html


Executing: 100%|██████████| 26/26 [00:18<00:00,  1.41cell/s]


HTML report saved for 100114M1.
Current data file: sub-10251061M_ses-01_task-VEP_run-1_eeg.edf
Participant ID: 10251061M
10251061M
Output notebook file: /project/def-emayada/q1k/experimental/hsj/derivatives/pylossless/derivatives/postproc/session_reports/executed_notebooks/10251061M_VEP_executed.ipynb
Output HTML file: /project/def-emayada/q1k/experimental/hsj/derivatives/pylossless/derivatives/postproc/session_reports/VEP/10251061M_VEP.html


Executing: 100%|██████████| 26/26 [00:15<00:00,  1.68cell/s]


HTML report saved for 10251061M.
Current data file: sub-100100F1_ses-01_task-VEP_run-1_eeg.edf
Participant ID: 100100F1
100100F1
Output notebook file: /project/def-emayada/q1k/experimental/hsj/derivatives/pylossless/derivatives/postproc/session_reports/executed_notebooks/100100F1_VEP_executed.ipynb
Output HTML file: /project/def-emayada/q1k/experimental/hsj/derivatives/pylossless/derivatives/postproc/session_reports/VEP/100100F1_VEP.html


...

HTML report saved for 10251061S3.
Current data file: sub-100128P_ses-01_task-VEP_run-1_eeg.edf
Participant ID: 100128P
100128P
Output notebook file: /project/def-emayada/q1k/experimental/hsj/derivatives/pylossless/derivatives/postproc/session_reports/executed_notebooks/100128P_VEP_executed.ipynb
Output HTML file: /project/def-emayada/q1k/experimental/hsj/derivatives/pylossless/derivatives/postproc/session_reports/VEP/100128P_VEP.html


Executing: 100%|██████████| 26/26 [00:15<00:00,  1.72cell/s]


HTML report saved for 100128P.
Current data file: sub-10086S1_ses-01_task-VEP_run-1_eeg.edf
Participant ID: 10086S1
10086S1
Output notebook file: /project/def-emayada/q1k/experimental/hsj/derivatives/pylossless/derivatives/postproc/session_reports/executed_notebooks/10086S1_VEP_executed.ipynb
Output HTML file: /project/def-emayada/q1k/experimental/hsj/derivatives/pylossless/derivatives/postproc/session_reports/VEP/10086S1_VEP.html


Executing: 100%|██████████| 26/26 [00:16<00:00,  1.61cell/s]


HTML report saved for 10086S1.
These subjects have errors: [] and need to be reprocessed
